# Practice

In [2]:
import numpy as np

## 1. Numpy Operation Axis

In [12]:
mat = np.array([[[1, 2, 3], [4, 5, 6]],[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]],[[1, 2, 3], [4, 5, 6]]])
mat

array([[[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]]])

In [13]:
mat.shape

(4, 2, 3)

In [14]:
mat.sum(axis=0)

array([[ 4,  8, 12],
       [16, 20, 24]])

In [15]:
mat.sum(axis=1)

array([[5, 7, 9],
       [5, 7, 9],
       [5, 7, 9],
       [5, 7, 9]])

In [16]:
mat.sum(axis=2)

array([[ 6, 15],
       [ 6, 15],
       [ 6, 15],
       [ 6, 15]])

In [17]:
mat.sum(axis=(0,1))

array([20, 28, 36])

In [9]:
mat.sum(axis=(0,2))

array([24, 42])

In [10]:
mat.sum(axis=(1,2))

array([21, 15, 15, 15])

In [18]:
mat.sum(axis=(0,1,2))

84

## 2. Model Load

In [1]:
import os
import sys
import pickle
import datetime
from glob import glob
from time import time
from tqdm.notebook import tqdm
from collections import Counter

# scikit-learn
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

# data processing
import cv2
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

# pytorch
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models, utils
torch.manual_seed(0)
print(f'Pytorch version: {torch.__version__}')

# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'This notebook use {device}')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Pytorch version: 1.6.0
This notebook use cuda:0


In [2]:
# 파일 경로 사용자 정의
class path:
    data = '../input/data'
    train = '../input/data/train'
    train_img = f'{train}/images'
    train_df = f'{train}/train.csv'
    test = '../input/data/eval'
    test_img = f'{test}/images'
    test_df = f'{test}/info.csv'

In [3]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [4]:
valid_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

In [5]:
submission = pd.read_csv(path.test_df)
image_dir = path.test_img
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, valid_transforms)
loader = DataLoader(dataset, shuffle=False)

In [6]:
def inference(model, test):
    model.eval()
    preds = []
    for images in tqdm(test):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            preds.extend(pred.cpu().numpy())
    submission['ans'] = preds
    now = datetime.datetime.now()
    submission.to_csv(f"{path.test}/submission_{now.strftime('%Y%m%d%H%M')}.csv")

In [8]:
model = models.resnet50(pretrained=False)
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, 18)
model = model.cuda()
model.load_state_dict(torch.load('./models/model2/epoch_8_cost_0.76_accr_0.96_f1_0.92.pt'))
inference(model, loader)